In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 43.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 51.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 4.4 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

In [ ]:
folderpath = '/content/drive/MyDrive/nlp-physicseducation/outputs/sections/'
filepath = 'labels_cleaned_y1y2.csv'

In [ ]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm  # progress bar
import tensorflow as tf

In [ ]:
df = pd.read_csv(folderpath+filepath)
df.head()

In [ ]:
# df.info()
# df['ArgumentLevel'].value_counts()

In [ ]:
df['ArgumentLevel'] = df['ArgumentLevel'].replace({'superficial': 0, 'extended': 1, 'deep': 2, 'expert': 3, 'prediction': 4})
# df['ArgumentLevel'] = df['ArgumentLevel'].replace({1: 0, 2: 1, 3: 2, 4: 3, 5: 4})
df['ArgumentLevel'].value_counts()

0    71
1    50
2    33
3    17
4     8
Name: ArgumentLevel, dtype: int64

In [ ]:
# df.head()

Tokenize one text

In [ ]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
# df['Content'].iloc[0]

In [ ]:
# token = tokenizer.encode_plus(
#     df['Content'].iloc[0],
#     max_length = 256,
#     truncation = True,
#     padding = 'max_length',
#     add_special_tokens = True, #add [CLS] [PAD] [SEP] tokens
#     return_tensors = 'tf'
# )

In [ ]:
# token

Tokenize all text

In [ ]:
X_input_ids = np.zeros((len(df), 256))
X_attn_masks = np.zeros((len(df), 256))

In [ ]:
# X_input_ids.shape

In [ ]:
def generate_training_data(df, ids, masks, tokenizer):
    for i, text in tqdm(enumerate(df['Content'])):
        tokenized_text = tokenizer.encode_plus(
            text,
            max_length = 256,
            truncation = True,
            padding = 'max_length',
            add_special_tokens = True, #add [CLS] [PAD] [SEP] tokens
            return_tensors = 'tf'
        )
        ids[i,:] = tokenized_text.input_ids
        masks[i,:] = tokenized_text.attention_mask
    return ids, masks

In [ ]:
X_input_ids, X_attn_masks = generate_training_data(df, X_input_ids, X_attn_masks, tokenizer)

0it [00:00, ?it/s]

Generate labels - One-hot encoding

In [ ]:
labels = np.zeros((len(df), 5)) # 5-classes label

"""
One-hot encoding
cat(0), dog(1), cow(2), chicken(3)
[
    [1, 0, 0, 0],
    [0, 1, 0, 0],
    [0, 0, 1, 0],
    [0, 0, 0, 1]
]
"""
# labels.shape

'\nOne-hot encoding\ncat(0), dog(1), cow(2), chicken(3)\n[\n    [1, 0, 0, 0],\n    [0, 1, 0, 0],\n    [0, 0, 1, 0],\n    [0, 0, 0, 1]\n]\n'

In [ ]:
labels[np.arange(len(df)), df['ArgumentLevel'].values] = 1

In [ ]:
# labels

Create and shuffle dataset

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((X_input_ids, X_attn_masks, labels))

In [ ]:
dataset.take(1)

<TakeDataset element_spec=(TensorSpec(shape=(256,), dtype=tf.float64, name=None), TensorSpec(shape=(256,), dtype=tf.float64, name=None), TensorSpec(shape=(5,), dtype=tf.float64, name=None))>

In [ ]:
def DatasetMapFunction(input_ids, attn_masks, labels):
    return {
        'input_ids': input_ids,
        'attention_mask': attn_masks
    }, labels

In [ ]:
dataset = dataset.map(DatasetMapFunction)

In [ ]:
# dataset.take(1)

In [ ]:
dataset = dataset.shuffle(400,reshuffle_each_iteration=True).batch(16, drop_remainder=True)
# Y1Y2 has 179 samples
# drop remainder 179/16 = 11...3, drop 3 data

Sample Training data & Validation data

In [ ]:
p = 0.5 # training data size 0.8
train_size = int((len(df)//16)*p)

In [ ]:
train_size  
# 8 batches, each batch has 16 data = 128 data
# the rest is validation data, = 11 - 8 = 3 batches

5

In [ ]:
train_dataset = dataset.take(train_size) # take the first 8 batches
val_dataset = dataset.skip(train_size) # take the last 3 batches

create model

In [ ]:
from transformers import TFBertModel
bert_model = TFBertModel.from_pretrained('bert-base-uncased')

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
# input layer
input_ids = tf.keras.layers.Input(shape = (256), name = 'input_ids', dtype = 'int32')
# attention layer
attention_masks = tf.keras.layers.Input(shape = (256), name = 'attention_mask', dtype = 'int32')

# bert embeddings - [0]: activation layer, [1]: pool layer 
bert_layer = bert_model.bert(input_ids, attention_mask=attention_masks)[1] 
# intermediate layer
intermediate_layer = tf.keras.layers.Dense(512, activation='relu', name = 'intermediate_layer')(bert_layer)
# output layer - 5 layers because 5 label classes
output_layer = tf.keras.layers.Dense(5, activation='softmax', name = 'output_layer')(intermediate_layer)

model = tf.keras.Model(inputs=[input_ids, attention_masks], outputs=output_layer)

model.layers[2].trainable = False # freeze bert layer for training if GPU is bad
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 256)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 256)]        0           []                               
                                                                                                  
 bert (TFBertMainLayer)         TFBaseModelOutputWi  109482240   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 256,                                           

Add optimizer & Loss function & Accuracy metrics

In [ ]:
optim = tf.keras.optimizers.Adam(learning_rate=1e-5, weight_decay=1e-6)
loss_func = tf.keras.losses.CategoricalCrossentropy()
acc = tf.keras.metrics.CategoricalAccuracy('accuracry')

In [ ]:
model.compile(optimizer = optim, loss = loss_func, metrics = [acc])

Train & Validate model (Fine-tuning)

In [ ]:
hist = model.fit(
    train_dataset,
    validation_data = val_dataset,
    epochs = 8 #20
)

Epoch 1/8
5/5 [==============================] - 10s 531ms/step - loss: 1.9065 - accuracry: 0.1000 - val_loss: 1.8763 - val_accuracry: 0.0833
Epoch 2/8
5/5 [==============================] - 0s 79ms/step - loss: 1.7921 - accuracry: 0.0875 - val_loss: 1.7544 - val_accuracry: 0.1979
Epoch 3/8
5/5 [==============================] - 0s 81ms/step - loss: 1.6885 - accuracry: 0.2000 - val_loss: 1.6997 - val_accuracry: 0.2083
Epoch 4/8
5/5 [==============================] - 0s 79ms/step - loss: 1.6406 - accuracry: 0.2000 - val_loss: 1.6452 - val_accuracry: 0.2188
Epoch 5/8
5/5 [==============================] - 0s 80ms/step - loss: 1.6450 - accuracry: 0.1625 - val_loss: 1.5601 - val_accuracry: 0.3021
Epoch 6/8
5/5 [==============================] - 0s 81ms/step - loss: 1.5698 - accuracry: 0.2875 - val_loss: 1.5230 - val_accuracry: 0.2812
Epoch 7/8
5/5 [==============================] - 0s 79ms/step - loss: 1.5251 - accuracry: 0.3875 - val_loss: 1.4947 - val_accuracry: 0.3646
Epoch 8/8
5/5 [===

Clear model after training

In [ ]:
tf.keras.backend.clear_session()
import gc
gc.collect()
del model

Save model

In [ ]:
model.save('Y1Y2_argument_model')

load model

In [ ]:
loaded_model = tf.keras.models.load_model('Y1Y2_argument_model')

Tokenise text of test data

In [ ]:
input_text = "this is abab bad babd superficial extended sentence. THink Think"

In [ ]:
def prepare_data(input_text, tokenizer):
    token = tokenizer.encode_plus(
        input_text,
        max_length = 256,
        truncation = True,
        padding = 'max_length',
        add_special_tokens = True, #add [CLS] [PAD] [SEP] tokens
        return_tensors = 'tf'
    )
    return {
        'input_ids': tf.cast(token.input_ids, tf.float64),
        'attention_mask': tf.cast(token.attention_mask, tf.float64)
    }

In [ ]:
tokenized_input_text = prepare_data(input_text, tokenizer)

In [ ]:
# tokenized_input_text

Test data

In [ ]:
probs = loaded_model.predict(tokenized_input_text)

1/1 [==============================] - 1s 1s/step


In [ ]:
# probs[0] # probability for five label classes
output_index = np.argmax(probs[0]) #max probability label

In [ ]:
classes = ['superficial','extended', 'deep', 'expert', 'prediction']

In [ ]:
print(classes[output_index])

extended
